In [1]:
import matplotlib.pyplot as plt
import numpy as np
import struct
from protozfits.simple import File
from pedestal import PedestalSimple, remove_pedestal
from event import Event
from tools import plot_hist

## Data model
|Header|| 
|------|---------------|
|      |version - unsigned char|
|      |number of pixels - unsigned short|
|      |number of samples - unsigned short|
|      |RoI -  	unsigned short|
|Data  |              |
|      | CH0 HG offset capacitor 0 - unsigned short|
|      | CH0 HG offset capacitor 1 - unsigned short|
|      | ... |
|      | CH0 HG offset capacitor 4095 - unsigned short|
|      | CH1 HG offset capacitor 0 - unsigned short|
|      | ... |
|      | CH7 HG (DRS _TAG) offset capacitor 4095 - unsigned short|
|---------------------------------------------|
|      | CH0 LG offset capacitor 0 - unsigned short|
|      | CH0 LG offset capacitor 1 - unsigned short|
|      | ... |
|      | CH0 LG offset capacitor 4095 - unsigned short|
|      | CH1 LG offset capacitor 0 - unsigned short|
|      | ... |
|      | CH7 LG (DRS _TAG) offset capacitor 4095 - unsigned short|

In [2]:
N1 = 20000
f1 = File("../data/Run021.1.fits.fz")
ped = PedestalSimple()
for i in range(0, N1):
    ev = next(f1.Events)
    Ev = Event(ev)
    Ev.read()
    ped.fill_pedestal_event(Ev)

ped.finalize_pedestal()
print("Finish calcurate pedestal")

Not enough events. Error:  invalid value encountered in sqrt
Finish calcurate pedestal


In [3]:

f_out = open('pedestal_value.dat', 'wb')

#header
f_out.write(struct.pack('>B', 1)) # version 1
f_out.write(struct.pack('>H', 8)) #number of pixels
f_out.write(struct.pack('>H', 4096)) #number of samples
f_out.write(struct.pack('>H', 40)) # RoI

#high gain
for i in range(0, 8):
    for value in (ped.mean_value_channel_high_gain[i, :]):
        f_out.write(struct.pack('>H', int(value)))
        
# low gain
for i in range(0, 8):
    for value in (ped.mean_value_channel_low_gain[i, :]):
        f_out.write(struct.pack('>H', int(value)))

f_out.close()        


with open("pedestal_value.dat", "rb") as binary_file:
    # Read the whole file at once
    data = binary_file.read()
    
    version = struct.unpack(">s", data[0:1])
    print("version: ",version)
    number_pixels = int.from_bytes(data[1:3], byteorder='big')
    print("number of pixels: ",number_pixels)
    number_samples = int.from_bytes(data[3:5], byteorder='big')
    print("number of samples: ",number_samples)
    RoI = int.from_bytes(data[5:7], byteorder='big')
    print("RoI: ", RoI)
    
    for i in range(5, 100, 2):
        value = int.from_bytes(data[i:i+2], byteorder='big')
        print(value)
        

version:  (b'\x01',)
number of pixels:  8
number of samples:  4096
RoI:  40
40
280
261
280
248
282
248
281
262
277
245
299
267
288
247
286
246
288
262
276
250
304
264
270
242
286
261
300
271
283
259
287
253
259
241
262
263
277
264
288
263
267
283
321
257
290
263
280
